# Part 1#

In [1]:
import pandas as pd
import numpy as np
import requests
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text

In [3]:
df = pd.read_html(source)[0]
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

In [4]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
df['Neighborhood'] = df['Neighborhood'].str.replace('/', ',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [5]:
df.shape

(103, 3)

## Part 2##

In [6]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
neighborhoods = pd.merge(df, coordinates, left_on='Postal code', right_on='Postal Code', how='left')
neighborhoods.drop('Postal Code', axis=1, inplace=True)
neighborhoods.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3 ##

In [8]:
toronto_neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighborhoods.head()


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Map of Toronto ####

In [9]:
#Toronto Coordinates 
Address = 'Toronto, Canada'
geolocator = Nominatim(user_agent='toronto_explorer')
coordinates = geolocator.geocode(Address)
coordinates = coordinates.latitude, coordinates.longitude
coordinates


(43.6534817, -79.3839347)

In [10]:
#Generate map
toronto_map = folium.Map(location = [coordinates[0],coordinates[1]] , zoom_start=11.5)

#add markers 
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    tooltip = '<p>'+label+'</p>'
    label = folium.Popup(label, parse_html=False)  
    folium.Marker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False,
        tooltip=tooltip).add_to(toronto_map)  


toronto_map

### Investigate venues ###

In [11]:
#Foursquare credentials
CLIENT_ID = '55QFNCVQ150GO1ZKVQXKQ2C0KE02KCCLZQOOWJQG43VZCHWV' # your Foursquare ID
CLIENT_SECRET = 'JLPJNHVVKRCURCXXMCRM3XF5TTWDSTPISY2TJ1X0A3US3LSC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 55QFNCVQ150GO1ZKVQXKQ2C0KE02KCCLZQOOWJQG43VZCHWV
CLIENT_SECRET:JLPJNHVVKRCURCXXMCRM3XF5TTWDSTPISY2TJ1X0A3US3LSC


In [12]:
#Coordinates for the 1st Neighborhood in the Toronto DataFrame

toronto_neighborhood_latitude = toronto_neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
toronto_neighborhood_longitude = toronto_neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               toronto_neighborhood_latitude, 
                                                               toronto_neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.6542599, -79.3606359.


In [13]:
#Get results for 1st neighborhood
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,toronto_neighborhoods.loc[0, 'Latitude'],toronto_neighborhoods.loc[0, 'Longitude'],500,LIMIT)

results = requests.get(url).json()

#JSON file for the 1st neighborhood
results

{'meta': {'code': 200, 'requestId': '5e8fd29229ce6a001b6d6e36'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [14]:
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '54ea41ad498e9a11e9e13308',
   'name': 'Roselle Desserts',
   'location': {'address': '362 King St E',
    'crossStreet': 'Trinity St',
    'lat': 43.653446723052674,
    'lng': -79.3620167174383,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383}],
    'distance': 143,
    'postalCode': 'M5A 1K9',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['362 King St E (Trinity St)',
     'Toronto ON M5A 1K9',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'grou

In [15]:
# copying function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # takes JSON and gives back a Dataframe

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


### Explore Neighborhoods ###

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Apply function above to the neighborhoods in Toronto

toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )


Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst  Quay , South Niagara , Island airport
Rosed

In [19]:
toronto_venues.shape

(1195, 7)

In [20]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [21]:
#Count the number of unique categories
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17,17,17,17,17,17
Central Bay Street,50,50,50,50,50,50
Christie,18,18,18,18,18,18
Church and Wellesley,50,50,50,50,50,50
"Commerce Court , Victoria Hotel",50,50,50,50,50,50
Davisville,37,37,37,37,37,37


In [22]:
#Number of distinct venue categories
len(toronto_venues['Venue Category'].unique())

216

### Analize Neighborhoods ###

In [23]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']]) # WHY????? 

toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

column_list = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[column_list]

toronto_onehot.head()

,Neighborhood,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,...,Venue Category_Thrift / Vintage Store,Venue Category_Toy / Game Store,Venue Category_Trail,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#grouping by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,...,Venue Category_Thrift / Vintage Store,Venue Category_Toy / Game Store,Venue Category_Trail,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.058824
3,"CN Tower , King and Spadina , Railway Lands , ...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.0,...,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.02,0.0,0.020000


In [25]:
toronto_grouped.shape

(39, 217)

In [26]:
#Define Function 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Cheese Shop,Venue Category_Farmers Market,Venue Category_Bakery,Venue Category_Seafood Restaurant,Venue Category_Restaurant,Venue Category_Beer Bar,Venue Category_Cocktail Bar,Venue Category_Liquor Store
1,"Brockton , Parkdale Village , Exhibition Place",Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Breakfast Spot,Venue Category_Performing Arts Venue,Venue Category_Bakery,Venue Category_Convenience Store,Venue Category_Pet Store,Venue Category_Climbing Gym,Venue Category_Restaurant,Venue Category_Burrito Place
2,Business reply mail Processing CentrE,Venue Category_Light Rail Station,Venue Category_Auto Workshop,Venue Category_Smoke Shop,Venue Category_Brewery,Venue Category_Burrito Place,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Restaurant,Venue Category_Recording Studio,Venue Category_Skate Park
3,"CN Tower , King and Spadina , Railway Lands , ...",Venue Category_Airport Service,Venue Category_Airport Lounge,Venue Category_Airport Terminal,Venue Category_Airport,Venue Category_Bar,Venue Category_Harbor / Marina,Venue Category_Rental Car Location,Venue Category_Sculpture Garden,Venue Category_Boutique,Venue Category_Boat or Ferry
4,Central Bay Street,Venue Category_Coffee Shop,Venue Category_Italian Restaurant,Venue Category_Spa,Venue Category_Burger Joint,Venue Category_Gym / Fitness Center,Venue Category_Japanese Restaurant,Venue Category_Café,Venue Category_Bubble Tea Shop,Venue Category_Hotel,Venue Category_Indian Restaurant


### Clustering Neighborhoods ###

In [28]:
# define number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0)

kmeans.fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 3, 3, 0, 3, 0, 3, 0])

In [29]:
# clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,3,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Bakery,Venue Category_Pub,Venue Category_Mexican Restaurant,Venue Category_Breakfast Spot,Venue Category_Café,Venue Category_Theater,Venue Category_Shoe Store,Venue Category_Restaurant
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,3,Venue Category_Coffee Shop,Venue Category_Diner,Venue Category_Yoga Studio,Venue Category_Park,Venue Category_Beer Bar,Venue Category_Boutique,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Sandwich Place,Venue Category_Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Ramen Restaurant,Venue Category_Cosmetics Shop,Venue Category_Italian Restaurant,Venue Category_Restaurant,Venue Category_Bookstore,Venue Category_Theater,Venue Category_Clothing Store,Venue Category_Burger Joint
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Bakery,Venue Category_Japanese Restaurant,Venue Category_Cocktail Bar,Venue Category_Farmers Market,Venue Category_Beer Bar,Venue Category_Park,Venue Category_Hotel,Venue Category_Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Venue Category_Trail,Venue Category_Pub,Venue Category_Neighborhood,Venue Category_Health Food Store,Venue Category_Yoga Studio,Venue Category_Cupcake Shop,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center


In [30]:
#1st Cluster - Dominated by cafes and coffeshops
toronto_merged[toronto_merged['Cluster Labels']==0]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Bakery,Venue Category_Japanese Restaurant,Venue Category_Cocktail Bar,Venue Category_Farmers Market,Venue Category_Beer Bar,Venue Category_Park,Venue Category_Hotel,Venue Category_Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Cheese Shop,Venue Category_Farmers Market,Venue Category_Bakery,Venue Category_Seafood Restaurant,Venue Category_Restaurant,Venue Category_Beer Bar,Venue Category_Cocktail Bar,Venue Category_Liquor Store
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Venue Category_Grocery Store,Venue Category_Café,Venue Category_Park,Venue Category_Candy Store,Venue Category_Baby Store,Venue Category_Athletics & Sports,Venue Category_Italian Restaurant,Venue Category_Diner,Venue Category_Coffee Shop,Venue Category_Nightclub
8,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568,0,Venue Category_Café,Venue Category_Restaurant,Venue Category_Coffee Shop,Venue Category_Bar,Venue Category_Asian Restaurant,Venue Category_American Restaurant,Venue Category_Steakhouse,Venue Category_Pizza Place,Venue Category_Gym,Venue Category_Gym / Fitness Center
9,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,0,Venue Category_Bakery,Venue Category_Pharmacy,Venue Category_Music Venue,Venue Category_Bar,Venue Category_Supermarket,Venue Category_Café,Venue Category_Bank,Venue Category_Middle Eastern Restaurant,Venue Category_Grocery Store,Venue Category_Brewery
11,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750,0,Venue Category_Bar,Venue Category_Restaurant,Venue Category_Vietnamese Restaurant,Venue Category_Asian Restaurant,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Men's Store,Venue Category_Yoga Studio,Venue Category_Italian Restaurant,Venue Category_Pizza Place
13,M5K,Downtown Toronto,"Toronto Dominion Centre , Design Exchange",43.647177,-79.381576,0,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Restaurant,Venue Category_Seafood Restaurant,Venue Category_Gastropub,Venue Category_Hotel,Venue Category_Beer Bar,Venue Category_Deli / Bodega,Venue Category_Bar,Venue Category_Japanese Restaurant
14,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,0,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Breakfast Spot,Venue Category_Performing Arts Venue,Venue Category_Bakery,Venue Category_Convenience Store,Venue Category_Pet Store,Venue Category_Climbing Gym,Venue Category_Restaurant,Venue Category_Burrito Place
16,M5L,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648198,-79.379817,0,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Seafood Restaurant,Venue Category_Hotel,Venue Category_Gastropub,Venue Category_Restaurant,Venue Category_Bakery,Venue Category_Deli / Bodega,Venue Category_Beer Bar,Venue Category_Gym
17,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Gastropub,Venue Category_Brewery,Venue Category_Bakery,Venue Category_American Restaurant,Venue Category_Yoga Studio,Venue Category_Comfort Food Restaurant,Venue Category_Sandwich Place,Venue Category_Cheese Shop


In [31]:
#2st Cluster - Gardens & Beauty
toronto_merged[toronto_merged['Cluster Labels']==1]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Venue Category_Garden,Venue Category_Health & Beauty Service,Venue Category_Yoga Studio,Venue Category_Cupcake Shop,Venue Category_Dumpling Restaurant,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center,Venue Category_Discount Store


In [32]:
#3rd Cluster - Outdoors
toronto_merged[toronto_merged['Cluster Labels']==2]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Venue Category_Trail,Venue Category_Pub,Venue Category_Neighborhood,Venue Category_Health Food Store,Venue Category_Yoga Studio,Venue Category_Cupcake Shop,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center
18,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Venue Category_Park,Venue Category_Swim School,Venue Category_Bus Line,Venue Category_Electronics Store,Venue Category_Dumpling Restaurant,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center,Venue Category_Discount Store
21,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,2,Venue Category_Park,Venue Category_Jewelry Store,Venue Category_Trail,Venue Category_Sushi Restaurant,Venue Category_Bus Line,Venue Category_Yoga Studio,Venue Category_Deli / Bodega,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run


In [33]:
#4th Cluster - Coffe & food
toronto_merged[toronto_merged['Cluster Labels']==3]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,3,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Bakery,Venue Category_Pub,Venue Category_Mexican Restaurant,Venue Category_Breakfast Spot,Venue Category_Café,Venue Category_Theater,Venue Category_Shoe Store,Venue Category_Restaurant
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,3,Venue Category_Coffee Shop,Venue Category_Diner,Venue Category_Yoga Studio,Venue Category_Park,Venue Category_Beer Bar,Venue Category_Boutique,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Sandwich Place,Venue Category_Café
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Ramen Restaurant,Venue Category_Cosmetics Shop,Venue Category_Italian Restaurant,Venue Category_Restaurant,Venue Category_Bookstore,Venue Category_Theater,Venue Category_Clothing Store,Venue Category_Burger Joint
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Venue Category_Coffee Shop,Venue Category_Italian Restaurant,Venue Category_Spa,Venue Category_Burger Joint,Venue Category_Gym / Fitness Center,Venue Category_Japanese Restaurant,Venue Category_Café,Venue Category_Bubble Tea Shop,Venue Category_Hotel,Venue Category_Indian Restaurant
10,M5J,Downtown Toronto,"Harbourfront East , Union Station , Toronto Is...",43.640816,-79.381752,3,Venue Category_Coffee Shop,Venue Category_Aquarium,Venue Category_Hotel,Venue Category_Plaza,Venue Category_Café,Venue Category_Bar,Venue Category_Park,Venue Category_Beer Bar,Venue Category_Dance Studio,Venue Category_Bistro
12,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,3,Venue Category_Greek Restaurant,Venue Category_Coffee Shop,Venue Category_Italian Restaurant,Venue Category_Restaurant,Venue Category_Ice Cream Shop,Venue Category_Bookstore,Venue Category_Furniture / Home Store,Venue Category_Grocery Store,Venue Category_Pizza Place,Venue Category_Brewery
15,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,3,Venue Category_Pizza Place,Venue Category_Park,Venue Category_Steakhouse,Venue Category_Sushi Restaurant,Venue Category_Ice Cream Shop,Venue Category_Fast Food Restaurant,Venue Category_Brewery,Venue Category_Fish & Chips Shop,Venue Category_Pub,Venue Category_Liquor Store
23,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Venue Category_Clothing Store,Venue Category_Coffee Shop,Venue Category_Sporting Goods Shop,Venue Category_Fast Food Restaurant,Venue Category_Diner,Venue Category_Dessert Shop,Venue Category_Mexican Restaurant,Venue Category_Park,Venue Category_Chinese Restaurant,Venue Category_Rental Car Location
24,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678,3,Venue Category_Sandwich Place,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Donut Shop,Venue Category_Cosmetics Shop,Venue Category_Burger Joint,Venue Category_Liquor Store,Venue Category_Indian Restaurant,Venue Category_History Museum,Venue Category_Middle Eastern Restaurant
25,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325,3,Venue Category_Gift Shop,Venue Category_Cuban Restaurant,Venue Category_Breakfast Spot,Venue Category_Dog Run,Venue Category_Italian Restaurant,Venue Category_Bar,Venue Category_Bank,Venue Category_Dessert Shop,Venue Category_Bookstore,Venue Category_Restaurant


In [34]:
#5th Cluster - Parks & Sports
toronto_merged[toronto_merged['Cluster Labels']==4]

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160,4,Venue Category_Park,Venue Category_Trail,Venue Category_Summer Camp,Venue Category_Tennis Court,Venue Category_Yoga Studio,Venue Category_Cupcake Shop,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center
33,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,4,Venue Category_Park,Venue Category_Playground,Venue Category_Trail,Venue Category_Cuban Restaurant,Venue Category_Donut Shop,Venue Category_Doner Restaurant,Venue Category_Dog Run,Venue Category_Distribution Center,Venue Category_Discount Store,Venue Category_Diner


In [35]:
#Visualize Clusters on the Map
# create map
map_clusters = folium.Map(location = [coordinates[0],coordinates[1]] , zoom_start=11.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters